<p>In a paper of Kahan (https://people.eecs.berkeley.edu/~wkahan/Qdrtcs.pdf) the issue of computing the quadratic equation arises.</p>

<p>Kahan write as $ax^2 + 2bx + c$ so the critical computation is the discriminant $b^2 - a \cdot c$. The issue comes from large enough values that the subtraction masks the low bits influence.</p>

<p>Here is an example in the paper, which should be 1.0 – not 2.0</p>

In [ ]:
a,b,c = 94906266.375, 94906267.375, 94906268.375
b^2 - a * c

2.0

<p>Dekker shows a way to split numbers into a high and low part:</p>

In [ ]:
function dekker_break(x)
    bigx = x * 134217729.0  # 2^27-1
    y = x - bigx
    xh = y + bigx
    xl = x - xh
    xh, xl
end

dekker_break (generic function with 1 method)

<p>We have</p>

In [ ]:
dekker_break(a)

(9.4906266e7, 0.375)

<p>Then we can compute <code>b^2</code> with:</p>

In [ ]:
bh, bl = dekker_break(b)
p = b*b
dp = ((bh * bh - p) + 2bh * bl) + bl * bl
p, dp

(9.00719958705499e15, -0.609375)

<p>And <code>a*c</code> is similar:</p>

In [ ]:
ah,al = dekker_break(a)
ch,cl = dekker_break(c)
q = a*c; dq = ((ah*ch - q) + (ah*cl + al*ch)) + al*cl
q, dq

(9.007199587054988e15, 0.390625)

<p>Here <code>p-q</code> gives the simple answer, the correction <code>dp-dq</code> carries the bits that were lost:</p>

In [ ]:
p-q, dp - dq

(2.0, -1.0)

<p>And then</p>

In [ ]:
(p-q) + (dp - dq)

1.0

<p>Note, we can check using BigFloats which have more precision:</p>

In [ ]:
big(b)^2 - big(a) * big(c)

1.000000000000000000000000000000000000000000000000000000000000000000000000000000

<p>With this, the quadratic equation can be solved with:</p>

In [ ]:
function discr(a, b, c)
    ah, al = dekker_break(a)  
    bh, bl = dekker_break(b)  
    ch, cl = dekker_break(c)  
    p = b*b
    dp = ((bh * bh - p) + 2bh * bl) + bl * bl
    q = a*c
	dq = ((ah*ch - q) + (ah*cl + al*ch)) + al*cl
    (p-q) + (dp - dq)
end

## Solve ax^2 + bx + c;  complex values not covered below
function qdrt(a, b, c)
#  b = -b
  d = discr(a,b,c)
  r = sqrt(d) * (sign(b) + iszero(b)) + b
  r/a, c/r
end

qdrt (generic function with 1 method)

<p>Trying it out we have:</p>

In [ ]:
a, b, c = 1, -1, 1 # x^2 - 2x + 1 -> (x-1)^2
qdrt(a, b, c)

(-1.0, -1.0)

<p>And</p>

In [ ]:
a,b,c = 94906266.375, 94906267.375, 94906268.375
qdrt(a, b, c)

(1.0000000210734241, 1.0)

<p>Which is correct, but other methods (this one using a linear algebra techique, not the straightforward discriminant calculution) can be wrong:</p>

In [ ]:
using Polynomials
x = variable()
r1, r2 = roots(a*x^2 - 2b*x + c)
r1, r2

(1.000000010536712, 1.000000010536712)